In [1]:
import pandas as pd
import os

os.chdir("./data")

In [2]:
df = pd.read_csv("E-Commerce_UK.csv")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,C-17850
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,C-17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,C-17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,C-17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,C-17850


#### 데이터 분할

In [5]:
refund_df = df.loc[df['Quantity'] <= 0] # 환불 데이터
order_df = df.loc[df['Quantity'] > 0] # 주문 데이터

#### 고객별 특징 추출

In [6]:
# 클러스터링을 위한 데이터 초기화
cluster_df = pd.DataFrame({'CustomerID':order_df['CustomerID'].unique()})

In [11]:
number_of_order_per_CID = order_df.drop_duplicates(subset=['CustomerID','InvoiceNo'])['CustomerID'].value_counts()

In [14]:
# 주문 횟수 계산 및 부착
# 중복제거 -> 한 주문에서 여러 상품을 주문하는 데이터이기 때문
number_of_order_per_CID = order_df.drop_duplicates(subset=['CustomerID','InvoiceNo'])['CustomerID'].value_counts()
cluster_df['주문횟수'] = cluster_df['CustomerID'].replace(number_of_order_per_CID.to_dict())

# 주문 횟수가 0인 경우에는 replace가 되지 않아 CustomerID가 부착될 수 있음
# 따라서 이러한 경우를 대비하기 위해 0으로 변경
cluster_df.loc[cluster_df['CustomerID']==cluster_df['주문횟수'], '주문횟수'] = 0
cluster_df.head()

,CustomerID,주문횟수
0,C-17850,34
1,C-13047,21
2,C-12583,15
3,C-13748,5
4,C-15100,4


In [19]:
# 반품 횟수 계산 및 부착
number_of_refund_per_CID = refund_df.drop_duplicates(subset=['CustomerID','InvoiceNo'])['CustomerID'].value_counts()
cluster_df['반품횟수'] = cluster_df['CustomerID'].replace(number_of_refund_per_CID.to_dict())

cluster_df.loc[cluster_df['CustomerID']==cluster_df['반품횟수'], '반품횟수'] = 0
cluster_df.head()

,CustomerID,주문횟수,반품횟수
0,C-17850,34,1
1,C-13047,21,8
2,C-12583,15,3
3,C-13748,5,0
4,C-15100,4,3


In [20]:
# 주문량 계산 및 부착
nubmer_of_quantity_per_CID = order_df.groupby('CustomerID')['Quantity'].sum()
cluster_df['주문량'] = cluster_df['CustomerID'].replace(nubmer_of_quantity_per_CID.to_dict())
cluster_df.loc[cluster_df['CustomerID']==cluster_df['주문량'], '주문량'] = 0
cluster_df.head()

,CustomerID,주문횟수,반품횟수,주문량
0,C-17850,34,1,1733
1,C-13047,21,8,1953
2,C-12583,15,3,5060
3,C-13748,5,0,439
4,C-15100,4,3,81


In [22]:
# 주문금액합계 계산 및 부착
order_df.loc[:,'주문금액'] = order_df.loc[:,'Quantity'] * order_df.loc[:,'UnitPrice']

number_of_quantity_per_CID = order_df.groupby('CustomerID')['주문금액'].sum()
cluster_df['주문금액합계'] = cluster_df['CustomerID'].replace(number_of_quantity_per_CID.to_dict())
cluster_df.loc[cluster_df['CustomerID']==cluster_df['주문금액합계'], '주문금액합계'] = 0
cluster_df.head()

C:\Users\hyun\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\hyun\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,CustomerID,주문횟수,반품횟수,주문량,주문금액합계
0,C-17850,34,1,1733,5391.21
1,C-13047,21,8,1953,6619.51
2,C-12583,15,3,5060,7281.38
3,C-13748,5,0,439,948.25
4,C-15100,4,3,81,877.25


In [23]:
# 최근 주문 - 현재 시점까지 거리 부착
current_date = pd.to_datetime('2011-12-10')
def recency(value):
    # InvoiceDate변수 참고 -> 공백을 기준으로 앞쪽은 날짜, 뒷쪽은 시간
    month, day, year = value.split(' ')[0].split('/') 
    diff = (current_date - pd.to_datetime('{}-{}-{}'.format(year, month, day))).days
    return diff
    
# keep = last(맨 뒤의 값 남김)를 통해, 계산량 감소 (날짜만 필요하니까 중복제거)
order_df_without_duplicates = order_df.drop_duplicates(subset=['CustomerID', 'InvoiceDate'], keep='last')[['CustomerID', 'InvoiceDate']]
order_df_without_duplicates['최근성'] = order_df_without_duplicates['InvoiceDate'].apply(recency)

min_recency_per_CID = order_df_without_duplicates.set_index('CustomerID')['최근성']
cluster_df['최근성'] = cluster_df['CustomerID'].replace(min_recency_per_CID.to_dict())

In [26]:
cluster_df.head()
# 일반적인 유클리디안, 맨하튼 거리를 쓰긴 부족
# 방향성만 보겠음 -> 코사인 유사도 확인
# 따라서 kmeans 사용하지 못하고 계층적군집화 사용

,CustomerID,주문횟수,반품횟수,주문량,주문금액합계,최근성
0,C-17850,34,1,1733,5391.21,373
1,C-13047,21,8,1953,6619.51,32
2,C-12583,15,3,5060,7281.38,3
3,C-13748,5,0,439,948.25,96
4,C-15100,4,3,81,877.25,331


In [29]:
# CustomerID는 군집화할 때 필요없는 정보이므로 index화
cluster_df.set_index(['CustomerID'], inplace=True)

#### 고객의 주문 특성에 따른 군집화 수행

In [30]:
# 군집화 모델 인스턴스화 및 학습
from sklearn.cluster import AgglomerativeClustering as AC

clustering_model = AC(n_clusters=5,
                     affinity='cosine',
                     linkage='average')
clustering_model.fit(cluster_df)

AgglomerativeClustering(affinity='cosine', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='average', memory=None, n_clusters=5)

In [32]:
cluster_df['주문특성_군집'] = clustering_model.labels_
cluster_df.head()

,주문횟수,반품횟수,주문량,주문금액합계,최근성,주문특성_군집
CustomerID,,,,,,
C-17850,34,1,1733,5391.21,373,0
C-13047,21,8,1953,6619.51,32,0
C-12583,15,3,5060,7281.38,3,0
C-13748,5,0,439,948.25,96,0
C-15100,4,3,81,877.25,331,0


In [33]:
# 군집별로 어떤 데이터가 모였는지
cluster_df.groupby(['주문특성_군집'])['주문횟수', '반품횟수', '주문량', '주문금액합계', '최근성'].mean()

,주문횟수,반품횟수,주문량,주문금액합계,최근성
주문특성_군집,,,,,
0,5.482738,1.075466,1502.736633,3073.161575,59.893064
1,3.370717,0.485981,1823.274143,1395.034455,69.040498
2,1.345733,0.188184,148.695842,272.678600,231.516411
3,1.086331,0.140288,59.593525,127.332122,272.694245
4,1.066667,0.266667,8.466667,27.213333,293.200000


#### 주요 상품 확인

In [34]:
order_df['Description'].value_counts().iloc[:100]

WHITE HANGING HEART T-LIGHT HOLDER    2323
JUMBO BAG RED RETROSPOT               2112
REGENCY CAKESTAND 3 TIER              2017
PARTY BUNTING                         1706
LUNCH BAG RED RETROSPOT               1594
                                      ... 
GUMBALL COAT RACK                      698
ANTIQUE SILVER T-LIGHT GLASS           696
SET OF 3 HEART COOKIE CUTTERS          696
JUMBO BAG SPACEBOY DESIGN              696
PLASTERS IN TIN SPACEBOY               689
Name: Description, Length: 100, dtype: int64

In [36]:
# 연관규칙탐색에서 배웠던 mlxtend
from mlxtend.preprocessing import TransactionEncoder
encoder = TransactionEncoder()

product_list_per_customer = order_df.groupby(['CustomerID'])['StockCode'].apply(list)

one_hot_df = encoder.fit(product_list_per_customer).transform(product_list_per_customer) #결과: ndarray
one_hot_df = pd.DataFrame(one_hot_df, columns=encoder.columns_, index=product_list_per_customer.index)
one_hot_df.head()
# 매우 희소

,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
CustomerID,,,,,,,,,,,,,,,,,,,,,
C-12346,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
C-12347,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
C-12348,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
C-12349,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
C-12350,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [39]:
# 최소 100히 이상 나온 품목만 사용
one_hot_df = one_hot_df.loc[:, one_hot_df.columns[one_hot_df.sum(axis=0) > 100]] # 열별 합계

In [43]:
clustering_model = AC(n_clusters=5,
                     affinity='jaccard', # 데이터가 희소하기때문에 자카드 유사도 사용
                     linkage='average')
clustering_model.fit(one_hot_df)

AgglomerativeClustering(affinity='jaccard', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='average', memory=None, n_clusters=5)

In [45]:
labels = clustering_model.labels_
one_hot_df['군집'] = labels

In [46]:
pd.Series(labels).value_counts()

# 군집화가 잘 안됐음
# 군집개수를 조정하든, 상품 구매 횟수 조정해서 보완

0    4305
1      36
3       1
2       1
4       1
dtype: int64